# OLD VERSION

In [ ]:
import pandas as pd
import os
from tqdm import tqdm, trange
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--data_type', default='index', help="Index or Time")
# parser.add_argument('--data_dir', default='data/', help="The path of data folder")

# args = parser.parse_args()

data_dir = '/tmp2/yhchen/Default_Prediction_Research_Project/data/8_labels_index/len_06/'
sample_data_dir = '/tmp2/yhchen/Default_Prediction_Research_Project/data/8_labels_index_sample/len_06/'
all_company_id_path = '/tmp2/yhchen/Default_Prediction_Research_Project/data/all_company_id.csv'


# compression = "gzip" if ".gz" in data_dir else None
compression = "gzip"


# all_company_id = pd.read_csv(all_company_id_path)
# company_sample_id = all_company_id.sample(n=500, random_state=1)

sample_train_df = pd.DataFrame()
sample_test_df = pd.DataFrame()

for i in trange(1, 13):
    # Join folder
    fold_dir = data_dir + "index_fold_{:02d}".format(i)
    sample_fold_dir = sample_data_dir + "index_fold_{:02d}".format(i)

    # Make folder if not exist
    if not os.path.isdir(sample_fold_dir):
        os.makedirs(sample_fold_dir)

    # Confirm load & save path
    test_path = os.path.join(fold_dir, 'test_cum.gz')
    train_path = os.path.join(fold_dir, 'train_cum.gz')
    sample_test_path = os.path.join(sample_fold_dir, 'test_cum.gz')
    sample_train_path = os.path.join(sample_fold_dir, 'train_cum.gz')

    # sklearn stratified sampling (default number or company number)

    # sampling
    train_df = pd.read_csv(train_path, compression=compression, header=0)
    test_df = pd.read_csv(test_path, compression=compression, header=0)

    sample_id = pd.DataFrame(train_df[train_df['id'].isin(test_df['id'])].id.unique(), columns=['id'])
    sample_id = sample_id.id.sample(n=500, random_state=1, replace = False)

    sample_train_df = train_df[train_df['id'].isin(sample_id)]
    sample_test_df = test_df[test_df['id'].isin(sample_id)]

    # write a pandas dataframe to gzipped CSV file
    sample_train_df.to_csv(sample_train_path, index=False, compression="gzip")
    sample_test_df.to_csv(sample_test_path, index=False, compression="gzip")

# NEW VERSION

In [ ]:
import pandas as pd

# The path of merged.csv
merge_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/merged.csv"
merge_df = pd.read_csv(merge_path, header=0)

In [ ]:
# Random Sampling

# Random Sampling

sample_id = pd.DataFrame(merge_df.id.unique(), columns=['id'])
sample_id = sample_id.id.sample(n=500, random_state=1, replace = False)

sample_merge_df = merge_df[merge_df['id'].isin(sample_id)]

# Save file
sample_merge_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/sample" # save path (folder)
# sample_merge_df.to_csv(sample_merge_path + "merged.csv", index=False)

In [ ]:
# Sampling according to distribution

# Sampling by distribution

b = merge_df.groupby('date')
default_list = list(b['y_cum_01'].sum())
time_list = list(b.size().index)

time_default = {}
for key, value in zip(time_list, default_list):
    time_default[key] = value

sample_id = pd.DataFrame(merge_df.id.unique(), columns=['id'])

id_default = {}
for id in sample_id.id:
    id_default[id] = 0

id_list = list(merge_df[merge_df['y_cum_01'] == 1]['id'])
time_list = list(merge_df[merge_df['y_cum_01'] == 1]['date'])
for key, time_idx in zip(id_list, time_list):
    id_default[key] = time_default[time_idx]

df_sample = pd.DataFrame(list(zip(list(id_default.keys()), list(id_default.values()))), columns =['id', 'weights'])

sample_id = df_sample.sample(n=500, random_state=1, replace = False, weights='weights')

sample_merge_df = merge_df[merge_df['id'].isin(sample_id.id)]

# Save file
sample_merge_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/sample/" # save path (folder)
# sample_merge_df.to_csv(sample_merge_path + "merged.csv", index=False)

# PLOT DISTRIBUTION

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sampling according to distribution
compression = "gzip"
train_path1 = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/merged.csv"

a = pd.read_csv(train_path1, header=0)
a['y_cum_01'] = a['y_cum_01'].replace({-1:0, 2:0})

b = a.groupby('date')
default_list = list(b['y_cum_01'].sum())
time_list = list(b.size().index)


df = pd.DataFrame(list(zip(time_list, default_list)), columns =['date', 'default_number'])
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [ ]:
import matplotlib.pyplot as plt

plt.bar(df.date, df.default_number, color = '#00A2FF', edgecolor="#00A2FF", label = "Actual", width=20, alpha=0.7)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Default numbers')
plt.show()

In [ ]:
sample_merge_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/sample/" # save path (folder)
a = pd.read_csv(sample_merge_path + "merged.csv", header=0)
a['y_cum_01'] = a['y_cum_01'].replace({-1:0, 2:0})
b = a.groupby('date')
default_list_sample = list(b['y_cum_01'].sum())
time_list_sample = list(b.size().index)

df_sample = pd.DataFrame(list(zip(time_list_sample, default_list_sample)), columns =['date', 'default_number'])
df_sample['date'] = pd.to_datetime(df_sample['date'],format='%Y-%m-%d')

import matplotlib.pyplot as plt

plt.bar(df.date, df.default_number, color = '#00A2FF', edgecolor="#00A2FF", label = "Actual", width=20, alpha=0.7)
plt.bar(df_sample.date, df_sample.default_number, color = 'red', edgecolor="red", label = "Sample", width=20, alpha=0.7)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Default numbers')
plt.show()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import relativedelta
import numpy as np

def plot_distribution(pred_path):
    # pred_path
    # pred_path = "/tmp2/cwlin/explainable_credit/explainable_credit/experiments/gru01_index/"
    pred_file = "pred.csv"

    print("Read merged.csv ...")
    merge_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/codes/data/interim/merged.csv"
    merge_df = pd.read_csv(merge_path, header=0, index_col=False)
    print("Read pred.csv ...")

    pred_df = pd.DataFrame()

    label_list = []
    pred_list = []

    # Read from 13 folds
    for i in range(1, 13):
        if i > 9:
            read_path = pred_path + 'index_fold_' + str(i) + '/' + pred_file
        else:
            read_path = pred_path + 'index_fold_0' + str(i) + '/' + pred_file
        df = pd.read_csv(read_path, header=0, index_col=False)
        pred_df = pd.concat([pred_df, df])

    # df = merge_df.merge(df, how ='inner', on =['date', 'id'])

    for i in range(1, 9):
        label_horizon = 'y_cum_0' + str(i)
        pred_horizon = 'p_cum_0' + str(i)

        # all_df = df.drop(index=df[df[pred_horizon] == -1].index)
        # all_df = all_df.drop(index=all_df[all_df[pred_horizon] == 2].index)
        all_df = pred_df
        all_df[pred_horizon] = all_df[pred_horizon].replace(-1, 0)
        all_df[pred_horizon] = all_df[pred_horizon].replace(2, 0)
        all_df = all_df.groupby('date')

        # df_label = merge_df.drop(index=merge_df[merge_df[label_horizon] == -1].index)
        # df_label = df_label.drop(index=df_label[df_label[label_horizon] == 2].index)
        df_label = merge_df
        df_label[label_horizon] = df_label[label_horizon].replace(-1, 0)
        df_label[label_horizon] = df_label[label_horizon].replace(2, 0)
        df_label = df_label.groupby('date')

        # label_list.append(list(all_df[label_horizon].mean()))
        # pred_list.append(list(all_df[pred_horizon].mean()))

        # label_list.append(list(all_df[label_horizon].sum()))
        # pred_list.append(list(all_df[pred_horizon].sum()))

        label_list = list(df_label[label_horizon].sum())
        pred_list = list(all_df[pred_horizon].sum())
        # pred_list = [x/13 for x in pred_list]
        time_list = list(all_df.size().index)

        print("Plot distribution of prediction and actual ...")
        df = pd.DataFrame(list(zip(time_list, pred_list, label_list)), columns =['date', 'pred_number', 'actual'])
        df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
        # df['date'] = df['date'] + relativedelta.relativedelta(months=predict_horizon-1)

        plt.bar(df.date, df.actual, color = '#00A2FF', edgecolor="#00A2FF", label = "Actual", width=20, alpha=0.7)
        plt.bar(df.date, df.pred_number, color = 'red', edgecolor="red", label = "Predict", width=20, alpha=0.7)
        plt.legend()
        plt.title('prediction horizon = ' + str(i))
        plt.xlabel('Year')
        plt.ylabel('Default numbers')
        plt.show()

plot_distribution(pred_path = "/tmp2/cwlin/explainable_credit/explainable_credit/experiments/gru01_index/")

Read merged.csv ...
Read pred.csv ...


<string>:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
<string>:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.


# Visualization

In [ ]:
from torchviz import make_dot
from model import *
from main import *

json_path = "/tmp2/yhchen/Default_Prediction_Research_Project/explainable_credit/experiments/gru06_index/index_fold_03/params.json"
params = Params(json_path)

params.batch_size = 1
params.num_stock = 15786
params.alpha = 0.2
params.heads_att = 6
params.hidn_att = 60
params.relation_static = 0

model = Neural_Default_Prediction_revised(params=params, adgat_relation=False)

x = torch.randn(6, 15786, 14).requires_grad_(True)  # 定义一个网络的输入值
y = model(x)    # 获取网络的预测值

MyConvNetVis = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
MyConvNetVis.format = "png"
# 指定文件生成的文件夹
MyConvNetVis.directory = "data"
# 生成文件
MyConvNetVis.view()

In [ ]:
from torchviz import make_dot
from model import *
from main import *

class test_model(nn.Module):
    def __init__(self):
        super(test_model, self).__init__()

        self.GRUs_r = Graph_GRUModel(num_nodes=15786, input_dim=14, hidden_dim=64)

        # self.gru_r = nn.GRU(input_size=14, hidden_size=64, batch_first=True)

    def reset_parameters(self):
        reset_parameters(self.named_parameters)

    def forward(self, x_r):
        # output, x_r = self.gru_r(input=x_r)
        x_r = self.GRUs_r(x_r)
        
        return x_r

model = test_model()

x = torch.randn(6, 15786, 14).requires_grad_(True)  # 定义一个网络的输入值
y = model(x)    # 获取网络的预测值

MyConvNetVis = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
MyConvNetVis.format = "png"
# 指定文件生成的文件夹
MyConvNetVis.directory = "data"
# 生成文件
MyConvNetVis.view()

In [ ]:
import hiddenlayer as h
from model import *
from main import *

class test_model(nn.Module):
    def __init__(self):
        super(test_model, self).__init__()

        self.gru_r = nn.GRU(input_size=14, hidden_size=64, batch_first=True)
        # self.GRUs_r = Graph_GRUModel(num_nodes=15786, input_dim=14, hidden_dim=64)

    def reset_parameters(self):
        reset_parameters(self.named_parameters)

    def forward(self, x_r):
        output, x_r = self.gru_r(input=x_r)
        # x_r = self.GRUs_r(x_r)
        
        return x_r

model = test_model()
vis_graph = h.build_graph(model, torch.zeros([6, 15786, 14]))   # 获取绘制图像的对象
vis_graph.theme = h.graph.THEMES["blue"].copy()     # 指定主题颜色
vis_graph.save("./demo1.png")   # 保存图像的路径

In [ ]:
model.named_parameters